# Heuristics - CVRP

[OR-Tools Documentation](https://or-tools.github.io/docs/pdoc/ortools/constraint_solver/pywrapcp.html)

To do:

- Implement skills
- Allow incomplete jobs

In [39]:
from itertools import cycle

import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
import matplotlib as mpl
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import functions as f
import importlib

## Instantiate data

In [40]:
n_vehicles = 5
seniority_multiplier = [1, 1, 1.2, 1.6, 1]
start_locations = [4, 5, 6, 7, 8]
end_locations = [0, 0, 0, 0, 0]

dataset = pd.read_csv("./data/custom_19.csv", index_col=0)
dataset_copy = dataset.copy()

for home_indx in start_locations:
    dataset.loc[home_indx, "timeslot"] = "AD"
    dataset.loc[home_indx, "job_type"] = "start"

coordinates = dataset.loc[:, ["x", "y"]]

N = coordinates.shape[0]

In [41]:
dataset

,x,y,timeslot,job_type
0,0,0,AD,end
1,-14,12,PM,DF-M Commision
2,-84,11,PM,DF-M Commision
3,-60,5,AM,DF-M Commision
4,79,64,AD,start
5,11,87,AD,start
6,8,68,AD,start
7,-4,24,AD,start
8,-11,20,AD,start
9,22,-5,AM,DF-M Exchange


In [42]:
# time windows, each each number represents 1 minute
time_window_dict = {
    "AD": [0, 510],  # 08:00 - 16:30
    "AM": [0, 240],  # 08:00 - 12:00
    "PM": [240, 510],  # 12:00 - 16:30
}

service_time_dict = {"start": 0, "DF-M Commision": 50, "DF-M Exchange": 100, "end": 0}

In [43]:
time_matrix_multiplier = 1  # used to make the drive times fill the day more

# assume travel time is proportional to euclidean distance
time_matrix = squareform(pdist(coordinates, metric="euclidean"))

# reduce the drive times to make feasible solutions more likely, will adjust later!
time_matrix = np.round(time_matrix * time_matrix_multiplier, decimals=0).astype(int)

time_matrix[:, 0] = time_matrix[0, :] = (
    0  # the end location is a dummy variable used to allow arbitrary end locations
)

## Model

In [44]:
# Create the routing index manager: number of nodes, number of vehicles, depot node
manager = pywrapcp.RoutingIndexManager(N, n_vehicles, start_locations, end_locations)

# Create Routing Model
routing = pywrapcp.RoutingModel(manager)

In [45]:
dataset

,x,y,timeslot,job_type
0,0,0,AD,end
1,-14,12,PM,DF-M Commision
2,-84,11,PM,DF-M Commision
3,-60,5,AM,DF-M Commision
4,79,64,AD,start
5,11,87,AD,start
6,8,68,AD,start
7,-4,24,AD,start
8,-11,20,AD,start
9,22,-5,AM,DF-M Exchange


### Parameters

In [46]:
# must create a different time_callback for
def create_time_callback(vehicle_id):
    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        travel_time = time_matrix[from_node, to_node]

        # convention is that job time added to drive FROM the job's node
        service_time = int(
            service_time_dict[dataset.loc[from_node, "job_type"]]
            * seniority_multiplier[vehicle_id]
        )

        return travel_time + service_time

    return time_callback


transit_callback_indices = []

for vehicle_id in range(n_vehicles):
    transit_callback = create_time_callback(vehicle_id)
    transit_callback_index = routing.RegisterTransitCallback(transit_callback)
    transit_callback_indices.append(transit_callback_index)

### Constraints

In [47]:
# Define time window contraints
time = "Time"

# create a CumulVar, for each location AND each vehicle
# 0 to N_locations-1, N_locations to N_locations+N_vehicles-1
# VERY IMPORTANT, the use index for cumVars, not the node numbers
routing.AddDimensionWithVehicleTransits(
    transit_callback_indices,
    1000,  # allow waiting time
    1000,  # maximum time per vehicle (and location but the range is set later)
    True,  # start cumul to zero
    time,
)

time_dimension = routing.GetDimensionOrDie(time)

# add time window constraints for each location via the index!
# cannot do for the depot as is special case
for location_indx in dataset.index:
    if location_indx == 0:
        continue
    elif location_indx in start_locations:
        continue

    timeslot = dataset.loc[location_indx, "timeslot"]

    index = manager.NodeToIndex(location_indx)

    time_dimension.CumulVar(index).SetRange(
        time_window_dict[timeslot][0],
        time_window_dict[timeslot][1],  # window depends on promised timeslot
    )

# Add max drive time for each vehicle
# do this by setting the range of the end node for each vehicle
for vehicle_id in range(n_vehicles):

    end_index = routing.End(vehicle_id)
    time_dimension.CumulVar(end_index).SetRange(0, 510)  # 8.5 hours max

    # ensures engineer start during work hours, not strictly necessary but worth keeping in
    start_index = routing.Start(vehicle_id)
    time_dimension.CumulVar(start_index).SetRange(0, 510)  # 8.5 hours max

# this for loop encourage the optimizer to minimse the time of the last job of each vehicle
for i in range(n_vehicles):

    # start as early as possible
    routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.Start(i)))

    # finish as early as possible
    routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

### Objective

In [49]:
# The cost of each arc is vehicle dependant!
for vehicle_id in range(n_vehicles):
    routing.SetArcCostEvaluatorOfVehicle(
        transit_callback_indices[vehicle_id], vehicle_id
    )

### Solution

In [50]:
# Setting heuristic strategies
search_parameters = pywrapcp.DefaultRoutingSearchParameters()

search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
)

search_parameters.time_limit.FromSeconds(5)

# Solve the problem
solution = routing.SolveWithParameters(search_parameters)

In [51]:
solution.ObjectiveValue()

1435

### Plot Results

In [75]:
calculated_objective_function_value = 0

print_val = "Key: |Node| A:Arrival, JT:Job Time, T:Travel Time, J:Job Time\n\n"
for vehicle_id in range(n_vehicles):
    index = routing.Start(vehicle_id)
    previous_index = None

    total_job_drive_time = 0
    total_drive_time = 0
    total_job_time = 0

    while not routing.IsEnd(index):

        previous_index = index
        previous_node = manager.IndexToNode(index)

        index = solution.Value(routing.NextVar(index))
        node = manager.IndexToNode(index)

        # arrival_time = solution.Min(time_dimension.CumulVar(previous_node))
        arrival_time = solution.Min(time_dimension.CumulVar(previous_index))

        job_drive_time = routing.GetArcCostForVehicle(previous_index, index, vehicle_id)

        total_job_drive_time += job_drive_time

        drive_time = time_matrix[previous_node][node]

        total_drive_time += drive_time

        job_time = service_time_dict[dataset.loc[previous_node, "job_type"]]

        total_job_time += job_time

        print_val += f"|{previous_node}| A:{arrival_time}, TS: {dataset.loc[previous_node, 'timeslot']}, JT:{job_drive_time}, T:{drive_time}, J:{job_time} -> "

    index = routing.End(vehicle_id)
    print_val += f"Finish: {solution.Value(time_dimension.CumulVar(index))} min\n"

    print_val += f"Totals: JT:{total_job_drive_time}, T:{total_drive_time}, J:{total_job_time}\n\n"

print(print_val)

print(f"Objective function value: {solution.ObjectiveValue()} min")
print(f"Ob function directly minimises JT, not finish time")

Key: |Node| A:Arrival, JT:Job Time, T:Travel Time, J:Job Time

|4| A:0, TS: AD, JT:36, T:36, J:0 -> |12| A:36, TS: AM, JT:127, T:27, J:100 -> |18| A:163, TS: AM, JT:159, T:59, J:100 -> |13| A:322, TS: PM, JT:50, T:0, J:50 -> Finish: 372 min
Totals: JT:372, T:122, J:250

|5| A:0, TS: AD, JT:90, T:90, J:0 -> |10| A:90, TS: AD, JT:56, T:6, J:50 -> |9| A:146, TS: AM, JT:216, T:116, J:100 -> |17| A:362, TS: AD, JT:50, T:0, J:50 -> Finish: 412 min
Totals: JT:412, T:212, J:200

|6| A:0, TS: AD, JT:46, T:46, J:0 -> |16| A:46, TS: AD, JT:60, T:0, J:50 -> Finish: 106 min
Totals: JT:106, T:46, J:50

|7| A:0, TS: AD, JT:16, T:16, J:0 -> |1| A:240, TS: PM, JT:80, T:0, J:50 -> Finish: 320 min
Totals: JT:96, T:16, J:50

|8| A:0, TS: AD, JT:51, T:51, J:0 -> |3| A:51, TS: AM, JT:91, T:41, J:50 -> |14| A:142, TS: AM, JT:51, T:1, J:50 -> |15| A:193, TS: AM, JT:120, T:20, J:100 -> |2| A:313, TS: PM, JT:86, T:36, J:50 -> |11| A:399, TS: AD, JT:50, T:0, J:50 -> Finish: 449 min
Totals: JT:449, T:149, J:300



In [72]:
372 + 412 + 106 + 96 + 449

1435

In [61]:
for location_indx in dataset.index:
    if location_indx == 0:
        continue
    timeslot = dataset.loc[location_indx, "timeslot"]
    time_var = time_dimension.CumulVar(manager.NodeToIndex(location_indx))
    print(
        f"Location {location_indx} ({timeslot}):  {solution.Value(time_var)}, {time_var}"
    )

print()
print("AM: [0, 240],  # 08:00 - 12:00 ")
print("PM: [240, 510],  # 12:00 - 16:30")

Location 1 (PM):  240, Time0(240..510)
Location 2 (PM):  313, Time1(240..510)
Location 3 (AM):  51, Time2(0..240)
Location 4 (AD):  0, Time3(0)
Location 5 (AD):  0, Time4(0)
Location 6 (AD):  0, Time5(0)
Location 7 (AD):  0, Time6(0)
Location 8 (AD):  0, Time7(0)
Location 9 (AM):  146, Time8(0..240)
Location 10 (AD):  90, Time9(0..510)
Location 11 (AD):  399, Time10(0..510)
Location 12 (AM):  36, Time11(0..240)
Location 13 (PM):  322, Time12(240..510)
Location 14 (AM):  142, Time13(0..240)
Location 15 (AM):  193, Time14(0..240)
Location 16 (AD):  46, Time15(0..510)
Location 17 (AD):  362, Time16(0..510)
Location 18 (AM):  163, Time17(0..240)

AM: [0, 240],  # 08:00 - 12:00 
PM: [240, 510],  # 12:00 - 16:30


In [64]:
449 + 96 + 106 + 412 + 372

1435

In [55]:
importlib.reload(f)
tours = f.compile_tours(n_vehicles, routing, manager, solution, True)

In [56]:
importlib.reload(f)
fig = f.plot_tours(tours, coordinates)
fig.show()

In [57]:
importlib.reload(f)
fig = f.plot_tours(tours, coordinates)
fig.show()